<a href="https://colab.research.google.com/github/fsdevrayan/pwa-chicken-breed-identification/blob/master/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
# configs
train_path = "/content/drive/MyDrive/chicken-breed-identification/dataset/train"
img_height, img_width= 224, 224
batch_size = 32
epochs = 10

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_path,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_path,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
num_classes = len(class_names)

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
def predict_image(img):
  img_4d=img.reshape(-1, 224, 224,3)
  prediction=model.predict(img_4d)[0]
  return {class_names[i]: float(prediction[i]) for i in range(len(class_names))}

In [ ]:
!pip install gradio
import gradio as gr

In [ ]:
image = gr.inputs.Image(shape=(224,224))
label = gr.outputs.Label()

gr.Interface(fn=predict_image, inputs=image, outputs=label,interpretation='default').launch(debug='True')

In [ ]:
!pip install tensorflowjs

model.save('/content/drive/MyDrive/chicken-breed-identification/dataset/main_model/keras_model.h5')
!tensorflowjs_converter --input_format=keras '/content/drive/MyDrive/chicken-breed-identification/dataset/main_model/keras_model.h5' '/content/drive/MyDrive/chicken-breed-identification/dataset/main_model'
print("Model saved and converted!")

In [ ]:
from tensorflow.keras import applications

mobileNetModel = applications.mobilenet.MobileNet()
mobileNetModel.save('/content/drive/MyDrive/chicken-breed-identification/dataset/mobilenet_model/keras_model.h5')
!tensorflowjs_converter --input_format=keras '/content/drive/MyDrive/chicken-breed-identification/dataset/mobilenet_model/keras_model.h5' '/content/drive/MyDrive/chicken-breed-identification/dataset/mobilenet_model'
print("Model saved and converted!")